### Install the latest .whl package

Check [here](https://pypi.org/project/semantic-link-labs/) to see the latest version.

In [ ]:
%pip install semantic-link-labs

### Requirements
* Must have an Azure Subscription
* Must [register an App](https://ms.portal.azure.com/#blade/Microsoft_AAD_RegisteredApps/ApplicationsListBlade)
    * Permissions: Will need the Contributor role at the scope where the resources will be created, which is often the subscription level
* Azure Key Vault
    * [Set up](https://learn.microsoft.com/azure/key-vault/secrets/quick-create-portal) within the Azure Subscription
    * Save secrets for the Tenant ID, Client ID (Application ID), Client Secret
    * Permissions: Ensure the user who will be executing the notebooks has “Key Vault Secrets User”
* Fabric Permissions
    * User should be a tenant admin. This ensures they have the necessary authority to execute and manage the notebooks without encountering permission issues.

### Result
* F skus are created for each (specified) capacity
    * Within the same region as the P SKU
    * Equivalent SKU size as the P SKU
    * Same admins as listed on the P SKU
    * All workspaces are migrated to the corresponding new capacity
* The names of the newly created F SKU capacities will be an alphanumeric lowercase version of the P SKU capacity name, suffixed with 'fsku'. As an example: "My capacity_3!" -> "mycapacity3fsku".


### Import the library and set the initial parameters

In [ ]:
import requests
from azure.identity import DefaultAzureCredential, ClientSecretCredential
from azure.keyvault.secrets import SecretClient
import os
import notebookutils
import pandas as pd

In [ ]:
# functions
def get_api_token_via_akv(kv_uri:str, client_id_secret:str, tenant_id_secret:str, client_secret_name:str)->str:
    """
    Function to retrieve an api token used to authenticate with Microsoft Fabric APIs

    kv_uri:str: The uri of the azure key vault
    client_id_secret:str: The name of the key used to store the value for the client id in the akv
    tenant_id_secret:str: The name of the key used to store the value for the tenant id in the akv
    client_secret_name:str: The name of the key used to store the value for the client secret in the akv

    """
    client_id = notebookutils.credentials.getSecret(kv_uri, client_id_secret)
    tenant_id = notebookutils.credentials.getSecret(kv_uri, tenant_id_secret)
    client_secret = notebookutils.credentials.getSecret(kv_uri, client_secret_name)

    credential = ClientSecretCredential(tenant_id, client_id, client_secret)
    scope = 'https://analysis.windows.net/powerbi/api/.default'
    token = credential.get_token(scope).token

    return token


def assign_workspace_to_capacity(workspace_list:list, capacity_str:str, api_token:str) ->None:
    """
    Given a list assign all workspaces in list to a capacity
    """
    for workspace_id in workspace_list:
        print(f'Attempting to migrate workspace:{workspace_id} to capacity:{capacity_str}')

        url = f'https://api.fabric.microsoft.com/v1/workspaces/{workspace_id}/assignToCapacity'

        post_body = {
            "capacityId": capacity_str
            }


        headers = {
        "Authorization": f"Bearer {api_token}",
        "Content-Type": "application/json"
        }    

        response = requests.post(url, headers=headers, json=post_body)
        print(response.status_code)


In [ ]:
kv_uri = 'https://kvfabricprodeus2rh.vault.azure.net/'
client_id_secret = 'fuam-spn-client-id'
tenant_id_secret = 'fuam-spn-tenant-id'
client_secret_name = 'fuam-spn-secret'

# get oauth token
token = get_api_token_via_akv(kv_uri, client_id_secret, tenant_id_secret, client_secret_name)

In [ ]:
workspace_list = ['2aeec6bf-64bd-4d25-be52-96846463b87c', 'fabe3025-1bb3-49a4-9948-09daf4e797c9', '79a33dbf-1742-40c5-885b-fb369152d26e']

In [ ]:
assign_workspace_to_capacity(workspace_list=workspace_list, capacity_str='76BFD071-AE1C-4189-8833-1A2F2B48E505', api_token=token)

### Setup for Key Vault Authentication

In [ ]:
import sempy_labs as labs
import sempy_labs.admin as admin

azure_subscription_id = '' # Enter your Azure subscription ID
resource_group = '' # Enter the name of the resource group (to be used to create the new F skus)

key_vault_uri = '', # Enter your key vault URI
key_vault_tenant_id = '', # Enter the key vault key to the secret storing your Tenant ID
key_vault_client_id = '', # Enter the key vault key to the secret storing your Client ID (Application ID)
key_vault_client_secret = '' # Enter the key vault key to the secret storing your Client Secret

### Migrate a single P SKU -> F SKU
Set the 'capacities' parameter to the single P SKU.

In [ ]:
with labs.service_principal_authentication(
    key_vault_uri=key_vault_uri, 
    key_vault_tenant_id=key_vault_tenant_id,
    key_vault_client_id=key_vault_client_id,
    key_vault_client_secret=key_vault_client_secret):

    labs.migrate_capacities(
        azure_subscription_id = azure_subscription_id,
        resource_group = resource_group,
        capacities = 'CapacityA',
        p_sku_only = True,
    )

### Migrate a list of P SKUs to F SKUs
Set the 'capacities' parameter to a list of P SKUs.

In [ ]:
with labs.service_principal_authentication(
    key_vault_uri=key_vault_uri, 
    key_vault_tenant_id=key_vault_tenant_id,
    key_vault_client_id=key_vault_client_id,
    key_vault_client_secret=key_vault_client_secret):

    labs.migrate_capacities(
        azure_subscription_id = azure_subscription_id,
        resource_group = resource_group,
        capacities = ['CapacityA', 'CapacityB', 'CapacityC'],
        p_sku_only = True,
    )

### Migrate all P SKUs to F SKUs
Set the 'capacities' parameter to None.

In [ ]:
with labs.service_principal_authentication(
    key_vault_uri=key_vault_uri, 
    key_vault_tenant_id=key_vault_tenant_id,
    key_vault_client_id=key_vault_client_id,
    key_vault_client_secret=key_vault_client_secret):

    labs.migrate_capacities(
        azure_subscription_id = azure_subscription_id,
        resource_group = resource_group,
        capacities = None,
        p_sku_only = True,
    )

### Migrate a list of P SKUs to F SKUs; associate each capacity with a specific resource group
This process ensures that each F SKU is created within the resource group specified in the resource_group_mapping dictionary.

In [ ]:
resource_group_mapping = {
    "CapacityA": "ResourceGroupA",
    "CapacityB": "ResourceGroupA",
    "CapacityC": "ResourceGroupB",
}

with labs.service_principal_authentication(
    key_vault_uri=key_vault_uri, 
    key_vault_tenant_id=key_vault_tenant_id,
    key_vault_client_id=key_vault_client_id,
    key_vault_client_secret=key_vault_client_secret):

    labs.migrate_capacities(
        azure_subscription_id = azure_subscription_id,
        resource_group = resource_group_mapping,
        capacities = ['CapacityA', 'CapacityB', 'CapacityC'],
        p_sku_only = True,
    )

### Migrate a single source capacity to a target capacity (target capacity already created)

In [ ]:
source_capacity = '' # Enter the Source capacity name
target_capacity = '' # Enter the Target capacity name (already exists) 

admin.assign_workspaces_to_capacity(
    source_capacity=source_capacity,
    target_capacity=target_capacity,
    workspace=None,
)

### Migrate a list of source capacities to target capacities (target capacity already created)

In [ ]:
capacity_mapping = {
    "capacitya": "capacityanew", # Format is "Source SKU": "Target SKU"
    "capacityb": "capacitybnew",
    "capacityc": "capacitycnew",
}

capacities = list(capacity_mapping.keys())

for capacity in capacities:
    admin.assign_workspaces_to_capacity(
        source_capacity=capacity,
        target_capacity=capacity_mapping.get(capacity),
        workspace=None,
    )

### Migrate a Fabric Trial Capacity (FT1) to a Fabric Capacity (F SKU)

In [ ]:
with labs.service_principal_authentication(
    key_vault_uri=key_vault_uri, 
    key_vault_tenant_id=key_vault_tenant_id,
    key_vault_client_id=key_vault_client_id,
    key_vault_client_secret=key_vault_client_secret):

    labs.migrate_fabric_trial_capacity(
        azure_subscription_id = azure_subscription_id,
        resource_group = resource_group,
        source_capacity = '', # The name of the Trial capacity.
        target_capacity = '', # The name of Fabric capacity (if it does not exist it will be created).
        target_capacity_sku = "F64", # Defaults to F64 but can specify the required SKU.
        target_capacity_admin_members = None, # Setting this to None will use the same admin members as the Trial capacity.
    )

# New Cells